### Установка PySpark

In [46]:
!pip install pyspark

### Создаем Spark сессию и устанавливаем зависимости

In [47]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("classification").getOrCreate()

### Имортирование дополнительных библиотек

In [48]:
from itertools import chain
from pyspark.sql.functions import count, mean, when, lit, create_map, regexp_extract

### Импортирование данных (Датасет Titanic)

In [49]:
df1 = spark.read.csv("tit_train.csv", header=True, inferSchema=True)
df2 = spark.read.csv("tit_test.csv", header=True, inferSchema=True)

### Просмотр схемы

In [50]:
df1.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [51]:
df1.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



### Отоборажение в Pandas

In [52]:
df1.limit(20).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,None,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,None,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,None,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,None,C


### Выборки

In [53]:
df1.select("Survived", "Pclass", "Age", "Fare").show(3)

+--------+------+----+-------+
|Survived|Pclass| Age|   Fare|
+--------+------+----+-------+
|       0|     3|22.0|   7.25|
|       1|     1|38.0|71.2833|
|       1|     3|26.0|  7.925|
+--------+------+----+-------+
only showing top 3 rows



### Описательные статистики

In [54]:
df1.select("Survived", "Pclass", "Age", "Fare").summary().show()

+-------+-------------------+------------------+------------------+-----------------+
|summary|           Survived|            Pclass|               Age|             Fare|
+-------+-------------------+------------------+------------------+-----------------+
|  count|                891|               891|               714|              891|
|   mean| 0.3838383838383838| 2.308641975308642| 29.69911764705882| 32.2042079685746|
| stddev|0.48659245426485753|0.8360712409770491|14.526497332334035|49.69342859718089|
|    min|                  0|                 1|              0.42|              0.0|
|    25%|                  0|                 2|              20.0|           7.8958|
|    50%|                  0|                 3|              28.0|          14.4542|
|    75%|                  1|                 3|              38.0|             31.0|
|    max|                  1|                 3|              80.0|         512.3292|
+-------+-------------------+------------------+------

In [55]:
print("Количество записей: \t", df1.count())
print("Количество переменных: \t", len(df1.columns))

Количество записей: 	 891
Количество переменных: 	 12


### EDA

### Сколько человек выживших

In [56]:
df1.groupBy("Survived").count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



### Количественные характеристики

In [57]:
df1.groupBy("Survived").mean("Fare", "Age").show()

+--------+------------------+------------------+
|Survived|         avg(Fare)|          avg(Age)|
+--------+------------------+------------------+
|       1| 48.39540760233917|28.343689655172415|
|       0|22.117886885245877| 30.62617924528302|
+--------+------------------+------------------+



### Факторные переменные

In [58]:
df1.groupBy("Survived").pivot("Sex").count().show()

+--------+------+----+
|Survived|female|male|
+--------+------+----+
|       1|   233| 109|
|       0|    81| 468|
+--------+------+----+



In [59]:
df1.groupBy("Survived").pivot("Pclass").count().show()

+--------+---+---+---+
|Survived|  1|  2|  3|
+--------+---+---+---+
|       1|136| 87|119|
|       0| 80| 97|372|
+--------+---+---+---+



In [60]:
df1.groupBy("Survived").pivot("SibSp").count().show()

+--------+---+---+---+---+---+----+----+
|Survived|  0|  1|  2|  3|  4|   5|   8|
+--------+---+---+---+---+---+----+----+
|       1|210|112| 13|  4|  3|NULL|NULL|
|       0|398| 97| 15| 12| 15|   5|   7|
+--------+---+---+---+---+---+----+----+



### Извлечение признаков

In [61]:
for col in df1.columns:
    print(col.ljust(20), df1.filter(df1[col].isNull()).count())

PassengerId          0
Survived             0
Pclass               0
Name                 0
Sex                  0
Age                  177
SibSp                0
Parch                0
Ticket               0
Fare                 0
Cabin                687
Embarked             2


In [62]:
df1.select("Fare", "Embarked").summary("mean", "50%", "max").show()

+-------+----------------+--------+
|summary|            Fare|Embarked|
+-------+----------------+--------+
|   mean|32.2042079685746|    NULL|
|    50%|         14.4542|    NULL|
|    max|        512.3292|       S|
+-------+----------------+--------+



### Извлечение данных о титулах

In [63]:
df1 = df1.withColumn("Title", regexp_extract(df1["Name"], "([A-Za-z]+)\.", 1))

df1.groupBy("Title").agg(count("Age"), mean("Age")).sort("count(Age)").show()

+--------+----------+------------------+
|   Title|count(Age)|          avg(Age)|
+--------+----------+------------------+
|     Don|         1|              40.0|
|Countess|         1|              33.0|
|    Lady|         1|              48.0|
|     Mme|         1|              24.0|
|    Capt|         1|              70.0|
|     Sir|         1|              49.0|
|Jonkheer|         1|              38.0|
|      Ms|         1|              28.0|
|     Col|         2|              58.0|
|    Mlle|         2|              24.0|
|   Major|         2|              48.5|
|     Rev|         6|43.166666666666664|
|      Dr|         6|              42.0|
|  Master|        36| 4.574166666666667|
|     Mrs|       108|35.898148148148145|
|    Miss|       146|21.773972602739725|
|      Mr|       398|32.368090452261306|
+--------+----------+------------------+



In [64]:
title_dic = {'Mr':'Mr', 'Miss':'Miss', 'Mrs':'Mrs', 'Master':'Master', \
             'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr',\
             'Don': 'Mr', 'Mme': 'Miss', 'Jonkheer': 'Mr', 'Lady': 'Mrs',\
             'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs', \
             'Dr':'Mr', 'Rev':'Mr'}

mapping = create_map([lit(x) for x in chain(*title_dic.items())])

df1 = df1.withColumn("Title", mapping[df1["Title"]])
df1.groupBy("Title").mean("Age").show()

+------+------------------+
| Title|          avg(Age)|
+------+------------------+
|  Miss|             21.86|
|Master| 4.574166666666667|
|    Mr| 33.02272727272727|
|   Mrs|35.981818181818184|
+------+------------------+



### Извлечение сведений по возрасту

In [65]:
def age_imputer(df, title, age):

    return df.withColumn('Age', when((df['Age'].isNull()) & (df['Title']==title), age).otherwise(df['Age']))

In [66]:
df1 = age_imputer(df1, "Mr", 33.02)
df1 = age_imputer(df1, "Mrs", 35.98)
df1 = age_imputer(df1, "Miss", 21.86)
df1 = age_imputer(df1, "Master", 4.75)

### Добавление агрегирующих жанных и удаление не используемых

In [67]:
df1 = df1.withColumn("FamilySize", df1["Parch"] + df1["SibSp"]).drop("Parch", "SibSp")

In [68]:
df1 = df1.drop("PassengerID", "Cabin", "Name", "Ticket", "Title")

In [69]:
df1.show(3)

+--------+------+------+----+-------+--------+----------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|FamilySize|
+--------+------+------+----+-------+--------+----------+
|       0|     3|  male|22.0|   7.25|       S|         1|
|       1|     1|female|38.0|71.2833|       C|         1|
|       1|     3|female|26.0|  7.925|       S|         0|
+--------+------+------+----+-------+--------+----------+
only showing top 3 rows



In [70]:
df1 = df1.fillna({"Embarked": "S"})

In [71]:
for col in df1.columns:
    print(col.ljust(20), df1.filter(df1[col].isNull()).count())

Survived             0
Pclass               0
Sex                  0
Age                  0
Fare                 0
Embarked             0
FamilySize           0


### Построение модели (Spark ML)

- StringIndexer: преобразование строковых категорий в численные
- Векторный Ассемблер: Spark API
- Логистическая регрессия на основе регуляризации гребня и Лассо
- Древовоидные анасамблевые методы: случайный лес градиентный спуск
- Конвейер

### Подключение библиотек из Spark ML

In [72]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

### String Indexer

In [73]:
stringIndex = StringIndexer(inputCols=["Sex", "Embarked"], outputCols=["SexNum", "EmbNum"])
stringIndex_model = stringIndex.fit(df1)
df1_ = stringIndex_model.transform(df1).drop("Sex", "Embarked")
df1_.show(3)

+--------+------+----+-------+----------+------+------+
|Survived|Pclass| Age|   Fare|FamilySize|SexNum|EmbNum|
+--------+------+----+-------+----------+------+------+
|       0|     3|22.0|   7.25|         1|   0.0|   0.0|
|       1|     1|38.0|71.2833|         1|   1.0|   1.0|
|       1|     3|26.0|  7.925|         0|   1.0|   0.0|
+--------+------+----+-------+----------+------+------+
only showing top 3 rows



### Векторный Ассемблер

In [74]:
vec_asmbl = VectorAssembler(inputCols=df1_.columns[1:], outputCol="features")
df1_ = vec_asmbl.transform(df1_).select("features", "Survived")
df1_.show(3, truncate=False)

+------------------------------+--------+
|features                      |Survived|
+------------------------------+--------+
|[3.0,22.0,7.25,1.0,0.0,0.0]   |0       |
|[1.0,38.0,71.2833,1.0,1.0,1.0]|1       |
|[3.0,26.0,7.925,0.0,1.0,0.0]  |1       |
+------------------------------+--------+
only showing top 3 rows



23/11/13 22:30:45 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 951586 ms exceeds timeout 120000 ms
23/11/13 22:30:45 WARN SparkContext: Killing executors is not supported by current scheduler.
23/11/13 22:47:18 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$